In [1]:
import pandas as pd
import torch
import torch.nn.functional as F

words = []
with open("./data/western_names.txt") as file:
    words = file.read().splitlines()

# with open("./data/indian_names.csv", encoding="utf-8") as file:
#     df = pd.read_csv(file, header=None)
#     words = df.iloc[:, 0].dropna().tolist()
#     words = [w.lower() for w in words if w.isalpha()]
#     words = words[1:]
words[0:10]
len(words)
min(len(w) for w in words)
max(len(w) for w in words)

15